In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d prasunroy/natural-images

 99% 340M/342M [00:17<00:00, 22.4MB/s]
100% 342M/342M [00:17<00:00, 20.4MB/s]


In [ ]:
!unzip natural-images.zip

Archive:  natural-images.zip
replace data/natural_images/airplane/airplane_0000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import cv2
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import RandomSampler

import torchvision.transforms as T
import torchvision.models as models
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder

from matplotlib import pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
import torch.optim as optim

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")
device = get_default_device()
device

device(type='cuda')

In [ ]:
classes = os.listdir('./natural_images')
num_classes = len(classes)
num_classes

8

In [ ]:
transform = transforms.Compose([
        # Randomly augment the image data
            # Random horizontal flip
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(0.5),
            # Random vertical flip
        transforms.RandomVerticalFlip(0.3),
        # transform to tensors
        transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

In [ ]:
batch_size = 50

all_data = datasets.ImageFolder(root='./natural_images')
train_data_len = int(len(all_data)*0.7)
test_data_len = int(len(all_data) - train_data_len)
train_data, test_data = random_split(all_data, [train_data_len, test_data_len])
train_data.dataset.transform = transform
test_data.dataset.transform = transform
print(len(train_data),len(test_data))

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

4829 2070


In [ ]:
# Get the spatial size of images
sample_image, _ = train_data[0]  # Retrieve the first image sample
image_size = sample_image.shape[1:]   # Get the size of the image

print("Spatial size of images: ", image_size)
num_channels = sample_image.shape[0]

print("Number of channels:", num_channels)

Spatial size of images:  torch.Size([256, 256])
Number of channels: 3


In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 12, 3, 1, 1)
    self.conv2 = nn.Conv2d(12, 24, 3, 1, 1)
    self.conv3 = nn.Conv2d(24, 48, 3, 1, 1)
    self.pool = nn.MaxPool2d(2, 2)
    self.drop = nn.Dropout2d(0.2)
    self.fc = nn.Linear(32*32*48, num_classes)

  def forward(self, x):
    x = F.relu(self.pool(self.conv1(x)))
    x = F.relu(self.pool(self.conv2(x)))
    x = F.relu(self.pool(self.conv3(x)))
    x = F.dropout(self.drop(x), training=self.training)
    x = x.view(-1, 32*32*48)
    x = self.fc(x)
    return torch.log_softmax(x, dim=1)


In [ ]:
model = Net()
model = model.to(device)
model.cuda()

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(24, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc): Linear(in_features=49152, out_features=8, bias=True)
)

In [ ]:
# Use an "Adam" optimizer to adjust weights
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Specify the loss criteria
criterion = nn.CrossEntropyLoss()

In [ ]:
# Define the number of epochs for training
num_epochs = 10
total_step = len(train_loader)
# Training loop
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    print('Epoch finished')

print('Training finished!')

Epoch [1/10], Step [1/97], Loss: 2.0812
Epoch [1/10], Step [2/97], Loss: 2.0555
Epoch [1/10], Step [3/97], Loss: 1.9362
Epoch [1/10], Step [4/97], Loss: 1.9079
Epoch [1/10], Step [5/97], Loss: 1.8729
Epoch [1/10], Step [6/97], Loss: 1.7775
Epoch [1/10], Step [7/97], Loss: 1.6285
Epoch [1/10], Step [8/97], Loss: 1.3489
Epoch [1/10], Step [9/97], Loss: 1.4812
Epoch [1/10], Step [10/97], Loss: 1.2851
Epoch [1/10], Step [11/97], Loss: 1.9442
Epoch [1/10], Step [12/97], Loss: 1.0734
Epoch [1/10], Step [13/97], Loss: 1.1978
Epoch [1/10], Step [14/97], Loss: 1.4695
Epoch [1/10], Step [15/97], Loss: 1.2128
Epoch [1/10], Step [16/97], Loss: 1.1296
Epoch [1/10], Step [17/97], Loss: 1.4060
Epoch [1/10], Step [18/97], Loss: 1.1174
Epoch [1/10], Step [19/97], Loss: 1.1718
Epoch [1/10], Step [20/97], Loss: 1.1002
Epoch [1/10], Step [21/97], Loss: 1.4183
Epoch [1/10], Step [22/97], Loss: 0.9467
Epoch [1/10], Step [23/97], Loss: 0.9739
Epoch [1/10], Step [24/97], Loss: 0.9558
Epoch [1/10], Step [25/97

In [ ]:
# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 88 %


In [ ]:
# alex net using transfer learning
alex= models.alexnet(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 215MB/s]


In [ ]:
# freez the model
for param in alex.parameters():
    param.requires_grad=False

In [ ]:
torch.manual_seed(42)

alex.classifier=nn.Sequential(  nn.Linear(9216,1024),
                                nn.ReLU(),
                                nn.Dropout(p=0.5),
                                nn.Linear(1024,len(classes)),
                                nn.LogSoftmax(dim=1))

In [ ]:
alex = alex.to(device)
alex.cuda()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, 

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(alex.classifier.parameters(),lr=0.001)

In [ ]:
num_epochs = 10
total_step = len(train_loader)
# Training loop
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = alex(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    print('Epoch finished')

print('Training finished!')

Epoch [1/10], Step [1/97], Loss: 2.0292
Epoch [1/10], Step [2/97], Loss: 1.5469
Epoch [1/10], Step [3/97], Loss: 1.5948
Epoch [1/10], Step [4/97], Loss: 1.4515
Epoch [1/10], Step [5/97], Loss: 0.8846
Epoch [1/10], Step [6/97], Loss: 0.5577
Epoch [1/10], Step [7/97], Loss: 0.7313
Epoch [1/10], Step [8/97], Loss: 0.1202
Epoch [1/10], Step [9/97], Loss: 0.3494
Epoch [1/10], Step [10/97], Loss: 0.0830
Epoch [1/10], Step [11/97], Loss: 0.4708
Epoch [1/10], Step [12/97], Loss: 0.1561
Epoch [1/10], Step [13/97], Loss: 0.3583
Epoch [1/10], Step [14/97], Loss: 0.4467
Epoch [1/10], Step [15/97], Loss: 0.2634
Epoch [1/10], Step [16/97], Loss: 0.4821
Epoch [1/10], Step [17/97], Loss: 0.1102
Epoch [1/10], Step [18/97], Loss: 0.7718
Epoch [1/10], Step [19/97], Loss: 0.4762
Epoch [1/10], Step [20/97], Loss: 0.7372
Epoch [1/10], Step [21/97], Loss: 0.3005
Epoch [1/10], Step [22/97], Loss: 0.0775
Epoch [1/10], Step [23/97], Loss: 0.4429
Epoch [1/10], Step [24/97], Loss: 1.1583
Epoch [1/10], Step [25/97

In [ ]:
# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = alex(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 98 %


In [ ]:
train_loader = DataLoader(train_data, batch_size=128, shuffle=False)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

In [ ]:
vgg = models.vgg16(pretrained=True)

In [ ]:
# freez the model
for param in vgg.parameters():
    param.requires_grad=False

In [ ]:
torch.manual_seed(42)
n_inputs = vgg.classifier[6].in_features
vgg.classifier[6]=nn.Sequential(  nn.Linear(n_inputs, 256),
                                nn.ReLU(),
                                nn.Dropout(p=0.4),
                                nn.Linear(256,len(classes)),
                                nn.LogSoftmax(dim=1))
print(vgg.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Sequential(
    (0): Linear(in_features=4096, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=256, out_features=8, bias=True)
    (4): LogSoftmax(dim=1)
  )
)


In [ ]:
from torch.optim import lr_scheduler
# Negative Log Likelihood Loss
criterion=nn.NLLLoss()
optimizer=torch.optim.Adam(vgg.classifier.parameters(),lr=0.0001)
# learning rate scheduler (adjusts the learning rate during training.)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
vgg = vgg.to(device)

In [ ]:
vgg.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
num_epochs = 10
total_step = len(train_loader)
# Training loop
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = vgg(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # exp_lr_scheduler.step()

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    print('Epoch finished')

print('Training finished!')

Epoch [1/10], Step [1/38], Loss: 2.1111
Epoch [1/10], Step [2/38], Loss: 2.0618
Epoch [1/10], Step [3/38], Loss: 1.9665
Epoch [1/10], Step [4/38], Loss: 1.8460
Epoch [1/10], Step [5/38], Loss: 1.8195
Epoch [1/10], Step [6/38], Loss: 1.7349
Epoch [1/10], Step [7/38], Loss: 1.6712
Epoch [1/10], Step [8/38], Loss: 1.6398
Epoch [1/10], Step [9/38], Loss: 1.5295
Epoch [1/10], Step [10/38], Loss: 1.4711
Epoch [1/10], Step [11/38], Loss: 1.3532
Epoch [1/10], Step [12/38], Loss: 1.3090
Epoch [1/10], Step [13/38], Loss: 1.2115
Epoch [1/10], Step [14/38], Loss: 1.2002
Epoch [1/10], Step [15/38], Loss: 1.0637
Epoch [1/10], Step [16/38], Loss: 1.0365
Epoch [1/10], Step [17/38], Loss: 1.0035
Epoch [1/10], Step [18/38], Loss: 0.9513
Epoch [1/10], Step [19/38], Loss: 0.8760
Epoch [1/10], Step [20/38], Loss: 0.7924
Epoch [1/10], Step [21/38], Loss: 0.7275
Epoch [1/10], Step [22/38], Loss: 0.6330
Epoch [1/10], Step [23/38], Loss: 0.6480
Epoch [1/10], Step [24/38], Loss: 0.6429
Epoch [1/10], Step [25/38

In [ ]:
# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 98 %
